In [1]:
import pandas as pd


df=pd.read_csv('df_sampled_가격2(edited for labeling) - df_sampled_가격2.csv')
df['polarity']=df['polarity'].str.upper()
df=df.drop(columns=['비고','기타','gpt_aspect','gpt_polarity','gpt_category','gpt_target'],errors='ignore')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     45 non-null     int64  
 1   product_name                   45 non-null     object 
 2   review                         45 non-null     object 
 3   aspect                         45 non-null     object 
 4   final_aspect                   0 non-null      float64
 5   polarity                       45 non-null     object 
 6   final_polarity                 0 non-null      float64
 7   category(가격, 배송, 품질, 디자인, 기타)  45 non-null     object 
 8   final_category                 0 non-null      float64
 9   수정 이유                          0 non-null      float64
 10  특이사항                           0 non-null      float64
dtypes: float64(5), int64(1), object(5)
memory usage: 4.0+ KB


In [2]:
df.head()

,product_id,product_name,review,aspect,final_aspect,polarity,final_polarity,"category(가격, 배송, 품질, 디자인, 기타)",final_category,수정 이유,특이사항
0,3370895800,로얄워터 블랑쉬 코튼 비누향 베이비파우더 살냄새 백화점 수제 승무원 장미 오드퍼퓸 ...,장미향을 찾고 있던중 여기 구매자수가 많아서 구매했는데 저랑은 향이 안맞네요. 빠른...,"만족도, 배송, 포장",NaN,"N, P, P",NaN,"품질, 배송, 배송",NaN,NaN,NaN
1,3370895800,로얄워터 블랑쉬 코튼 비누향 베이비파우더 살냄새 수제 승무원 엑스트레 드 퍼퓸 30ml,너무실망했어요.베이비파우더향이너무진하고향이너무인이적이고반품하고싶어요.아주별루예요.,"만족도, 농도, 향조",NaN,"N, N, N",NaN,"품질, 품질, 품질",NaN,NaN,NaN
2,3370895800,로얄워터 블랑쉬 코튼 비누향 베이비파우더 살냄새 백화점 수제 승무원 장미 오드퍼퓸 ...,처음에 선물 받아서 로얄워터 제품을 접하고 만족해서 교보에서 몇 번 더 구매했습니다...,"톡 쏘는 향, 알코올 향, 향조, 만족도, 추천",NaN,"N, N, N, N, N, N",NaN,"품질, 품질, 품질, 품질, 품질",NaN,NaN,NaN
3,3370895800,로얄워터 블랑쉬 코튼 비누향 베이비파우더 살냄새 수제 승무원 엑스트레 드 퍼퓸 30ml,갑자기 구매 금액이 너무 비싸졌습니다.,가격,NaN,N,NaN,가격,NaN,NaN,NaN
4,3370895800,로얄워터 블랑쉬 코튼 비누향 베이비파우더 살냄새 백화점 수제 승무원 장미 오드퍼퓸 ...,3개 구매 하면 1개 렌덤 이라더니 작은샘플 하나 주네요 처음 부터 샘플 준다고 하...,"판매 방식, 추천",NaN,"N, N",NaN,"서비스운영, 품질",NaN,NaN,NaN


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
template='''
다음은 리뷰에 대해 사람이 만든 ABSA 분석 라벨입니다. 해당 라벨을 검토하고, 필요하다면 수정해주세요.

#### 입력:
상품명: {product_name}
리뷰: {review}
1. 사람이 만든 aspect 라벨: {aspect}
2. 사람이 만든 polarity 라벨: {polarity}
3. 사람이 만든 category 라벨: {category}

#### 수정된 라벨과 간단한 이유를 아래 json 형식으로 출력해주세요:
   "gpt_aspect": "", (수정된 aspect 라벨, 바뀐 라벨을 제외한 기존 라벨)
   "gpt_polarity": "", (수정된 aspect 라벨, 바뀐 라벨을 제외한 기존 라벨)
   "gpt_category": "", (수정된 aspect 라벨, 바뀐 라벨을 제외한 기존 라벨)
   "수정 이유": ""라벨 수정 또는 유지에 대한 간단한 이유

#### 라벨링 기준:
1. aspect: 제품(entity) 혹은 제품의 속성(attribute)을 나타내는 명사 또는 명사구
2. polarity: aspect에 대한 P, N, O(중립)으로 나타내세요.
3. category: 배송, 가격, 디자인, 품질, 서비스운영으로 나타내세요.
   - 배송: 고객이 받은 배송 경험과 관련된 평가를 의미.
   - 가격: 고객이 제품이나 서비스의 비용 대비 가치를 어떻게 느끼는지에 대한 평가.
   - 디자인: 고객이 받은 상품이나 서비스의 외형적 아름다움, 실용성, 사용자 경험, 무게, 외형을 포함한 시각적 요소에 대한 평가.
   - 품질: 고객 경험에 의해서 고객이 받은 전반적인 상품이나 서비스의 기능 및 성능이 어느정도로 기대에 부합하는가에 대한 개념.
   - 서비스운영: 고객이 긍정적인 경험을 할 수 있도록 서비스를 설계하고 실행하며, 문제를 신속하고 체계적으로 해결하는 과정.

'''

In [5]:
prompt=PromptTemplate.from_template(template=template)
model=ChatOpenAI()
output_parser=JsonOutputParser()
chain=prompt|model|output_parser

In [6]:
results=[]

for i in range(len(df)):
    product_name = df['product_name'].iloc[i]
    review = df['review'].iloc[i]
    aspect=df['aspect'].iloc[i]
    polarity=df['polarity'].iloc[i]
    category=df['category(가격, 배송, 품질, 디자인, 기타)'].iloc[i]

    print(f'{i}번째 - 리뷰: {review}')
    output=chain.invoke({'product_name':f'{product_name}','review':f'{review}','aspect':f'{aspect}','category':f'{category}','polarity':f'{polarity}'})

    results.append({
        'gpt_aspect': output.get('gpt_aspect'),
        'gpt_polarity':output.get('gpt_polarity'),
        'gpt_category':output.get('gpt_category'),
        '수정 이유':output.get('수정 이유')
    })

gpt_results_df=pd.DataFrame(results)
final_df=pd.concat([df.reset_index(drop=True),gpt_results_df],axis=1)
final_df.info()

0번째 - 리뷰: 장미향을 찾고 있던중 여기 구매자수가 많아서 구매했는데 저랑은 향이 안맞네요. 빠른배송이나 포장상태는 좋습니다.
1번째 - 리뷰: 너무실망했어요.베이비파우더향이너무진하고향이너무인이적이고반품하고싶어요.아주별루예요.
2번째 - 리뷰: 처음에 선물 받아서 로얄워터 제품을 접하고 만족해서 교보에서 몇 번 더 구매했습니다(선물용으로도 좋아서) 교보 제품들은 만족스러웠어요 그래서 이번에 네이버에서 블루밍을 구매해봤는데 다른 제품이 배송된 게 아닐까 싶을 정도로 톡 쏘는 향이 너무 강했고 알콜이 그대로 느껴졌어요 바로 컴플레인을 할까 생각했지만 숙성기간이 지나야한다라고 하실 것 같아 그냥 시간이 지나기만 기다렸습니다 그런데 한달이 지났는데도 후추향같은 톡 쏘는 향만 강하고 머리 아픈 진한 꽃냄새만 납니다 여리여리한 블루밍의 매력은 아예 없어요 제가 받은 제품에 문제가 있는 건지 로얄워터의 블루밍이 원래 이런 건지 알 수가 없으니 답답하고 실망스럽습니다 이럴 줄 알았으면 받자마자 교환이든 환불이든 받을 걸 괜히 기다렸단 후회밖에 안드네요 안좋은 후기 쓸 바엔 다신 그 상품을 쓰지말자 주의덴 다른 로얄워터 제품들은 만족했던 터라 한달후기 씁니다
3번째 - 리뷰: 갑자기 구매 금액이 너무 비싸졌습니다.
4번째 - 리뷰: 3개 구매 하면 1개 렌덤 이라더니 작은샘플 하나 주네요 처음 부터 샘플 준다고 하지 비추천 입니다
5번째 - 리뷰: 예전부터 꾸준히 구매해왔고 이후에 쿠*서도 계속 구매중이었습니다 20000원까지 오를때도 급격하다 생각하면서도 이해했지만 패키지 교체로 25000원으로 오른건 좀 심하네요 솔직히 오리지널리티가 있는 것도 아닌데 이걸 누가 선물을 해요 저렴이로 자기가 쓰는거지 그리고 패키지 값이 5천원요??? 원자재값이 올라서 또 올리는거면 이해를 하죠 패키지 핑계로 5천원이라.. 양심 좀 가격이 저렴해서 신제품 나올때마다 이것저것 사보곤 했는데 이젠 그럴 일이 없을듯요 이번엔 쿠폰으로 인상전과 별차이가 없어 사봤는데 73번 별롭니다 뭔가 남자스

In [7]:
final_df.to_csv('final_df_가격2.csv',index=False,encoding='utf-8')